#  Get data ready

In [3]:
import pandas as pd

import torch
import torchvision as tv
from torchvision import datasets, transforms
from torch.utils.data import DataLoader as DL

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.utils import resample

In [4]:
#import of review data
cols = ["user id","item id","rating","timestamp"]
#encoding using ISO-8859-1 is used because utf-8 does not support all the characters in movie names
df_data = pd.read_csv("ml-100k/u.data",sep="\t",names=cols,header=None,encoding="ISO-8859-1")

In [5]:
#verifying the sucessful import of review data
df_data.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [16]:
#import of moviedata
cols = ["movie id","movie title","release date","video release date","IMDb URL","unknown",
        "Action","Adventure","Animation","Childrens","Comedy","Crime","Documentary",
        "Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi",
        "Thriller","War","Western"]

df_movie = pd.read_csv("ml-100k/u.item",sep="|",names=cols,header=None,encoding="ISO-8859-1")

In [18]:
df_movie[['Action', 'Adventure',
       'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western']].sum()

Action         251
Adventure      135
Animation       42
Childrens      122
Comedy         505
Crime          109
Documentary     50
Drama          725
Fantasy         22
Film-Noir       24
Horror          92
Musical         56
Mystery         61
Romance        247
Sci-Fi         101
Thriller       251
War             71
Western         27
dtype: int64

In [7]:
#verifying the sucessful import of movie data
df_movie.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
#import of user data
cols = ["user id","age","gender","occupation","zip code"]
df_user = pd.read_csv("ml-100k/u.user",sep="|",names=cols,header=None,encoding="ISO-8859-1")

In [9]:
#verifying the sucessful import of user data
df_user.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [10]:
#frequency binning the ages into age groups as it will be easier for future analysis
df_user['age_group'] = pd.qcut(df_user['age'],q=10,precision=0)

#the bins are of unequal size due to repeating values in a bin
df_user['age_group'].value_counts()

(6.0, 20.0]     109
(23.0, 26.0]    105
(35.0, 40.0]    100
(31.0, 35.0]     98
(29.0, 31.0]     96
(40.0, 46.0]     94
(46.0, 51.0]     93
(20.0, 23.0]     92
(51.0, 73.0]     85
(26.0, 29.0]     71
Name: age_group, dtype: int64

In [11]:
#join all three dataframes

df = pd.merge(pd.merge(df_data,
                  df_user[["user id",
                           "age_group",
                           "gender",
                           "occupation"]],
                  on='user id',
                  how='left'),
              df_movie,
              left_on = 'item id',
              right_on = 'movie id',
              how ='left')

In [12]:
#verify join
df.head()

,user id,item id,rating,timestamp,age_group,gender,occupation,movie id,movie title,release date,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,"(46.0, 51.0]",M,writer,242,Kolya (1996),24-Jan-1997,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,"(35.0, 40.0]",F,executive,302,L.A. Confidential (1997),01-Jan-1997,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,"(23.0, 26.0]",M,writer,377,Heavyweights (1994),01-Jan-1994,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,"(26.0, 29.0]",M,technician,51,Legends of the Fall (1994),01-Jan-1994,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,"(46.0, 51.0]",M,educator,346,Jackie Brown (1997),01-Jan-1997,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#drop unneccessary features
df.drop(["movie id",
        "movie title",
        "release date",
        "video release date",
        "IMDb URL",
        "unknown",
        "user id",
        "item id",
        "timestamp"],axis=1, inplace=True)

In [211]:
#verify drop
df.columns

Index(['rating', 'age_group', 'gender', 'occupation', 'Action', 'Adventure',
       'Animation', 'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [212]:
#categorize age_group, gender and occupation using 1-hot encoder
df['age_group'] = pd.Categorical(df['age_group'])
df['gender'] = pd.Categorical(df['gender'])
df['occupation'] = pd.Categorical(df['occupation'])

age_group_dummies = pd.get_dummies(df['age_group'])
gender_dummies = pd.get_dummies(df['gender'])
occupation_dummies = pd.get_dummies(df['occupation'])

df = pd.concat([df,
                age_group_dummies,
                gender_dummies,
                occupation_dummies], axis=1)

df.drop(['age_group',
        'gender',
        'occupation'], axis=1, inplace=True)

In [213]:
#verify categorization
df.head()

,rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,...,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer
0,3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Neural Network Part 

In [214]:
#create class for the neural network

'''
fully connected layer = fc
nn.Linear(input, ouput)
initial input =  number of columns = 51
middle layers = 3 layers of 64 neurons
final output = number of ratings (0-5) = 6
'''

class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(51, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 6)

#ReLU activation function on hidden layers
#Use log_softmax for output to get probability for classes
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        
        return F.log_softmax(x, dim=1)
    
#view created network

net = Net()
net

Net(
  (fc1): Linear(in_features=51, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=6, bias=True)
)

In [215]:
#check count of ratings
df.rating.value_counts()

4    34174
3    27145
5    21201
2    11370
1     6110
Name: rating, dtype: int64

In [216]:
#balance the testing ratings using downsampling
'''
ns = 6000
df_1 = df[df.rating == 1]
df_2 = df[df.rating == 2]
df_3 = df[df.rating == 3]
df_4 = df[df.rating == 4]
df_5 = df[df.rating == 5]

#downsample all classes except 1 to 6110
df_2_downsampled = resample(df_2,
                                 replace = False,
                                 n_samples = ns)
df_3_downsampled = resample(df_3,
                                 replace = False,
                                 n_samples = ns)
df_4_downsampled = resample(df_4,
                                 replace = False,
                                 n_samples = ns)
df_5_downsampled = resample(df_5,
                                 replace = False,
                                 n_samples = ns)

df = pd.concat([df_1, df_2_downsampled,
                     df_3_downsampled,
                     df_4_downsampled,
                     df_5_downsampled])
'''

ns = 25000
df_1 = df[df.rating == 1]
df_2 = df[df.rating == 2]
df_3 = df[df.rating == 3]
df_4 = df[df.rating == 4]
df_5 = df[df.rating == 5]

#downsample all classes to 30,000
df_1_upsampled = resample(df_1,
                                 replace = True,
                                 n_samples = ns,
                                 random_state=123)
df_2_upsampled = resample(df_2,
                                 replace = True,
                                 n_samples = ns,
                                 random_state=123)
df_3_upsampled = resample(df_3,
                                 replace = True,
                                 n_samples = ns,
                                 random_state=123)
df_4_upsampled = resample(df_4,
                                 replace = True,
                                 n_samples = ns,
                                 random_state=123)
df_5_upsampled = resample(df_5,
                                 replace = True,
                                 n_samples = ns,
                                 random_state=123)

df = pd.concat([df_1_upsampled,
                df_2_upsampled,
                df_3_upsampled,
                df_4_upsampled,
                df_5_upsampled])

df.rating.value_counts()

5    25000
4    25000
3    25000
2    25000
1    25000
Name: rating, dtype: int64

In [217]:
#resetting the index
df.reset_index(inplace = True, drop = True)

In [218]:
#prepare data for PyTorch
rank = df['rating'].values
df_input = df.drop(["rating"], axis=1)

i = []

for index,row in df_input.iterrows():
    t = (torch.tensor(row.values), rank[index])
    i.append(t)
i = tuple(i)
i

((tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]), 1),
 (tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
          1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1]),
  1),
 (tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
          0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          1, 0, 0]), 1),
 (tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 1]), 1),
 (tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
          0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          1, 0, 0]), 1),
 (tensor([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

In [219]:
#split into train and test 80%-20%
slice = int(0.8*len(i))
train = i[:slice]
test = i[slice:]
len(test)

25000

In [220]:
#divide into batches
'''
batch_size = how many inputs to pass to model at a time
shuffle = to shuffle inputs or not
'''
trainset = DL(train, batch_size=32, shuffle=True)
testset = DL(test, batch_size=32, shuffle=True)

In [221]:
#lr = learning rate = 0.001
opt = optim.Adam(net.parameters(), lr = 0.001)

#EPOCHS = number of times to iterate over dataset
EPOCHS = 5

#train the network
'''
loss = error
zero_grad() = makes gradient zero after batch
nll_loss = calculates loss to update weights
if data is 1 hot vector, use mean squared error
backward() = propogate the weights backward
opt.step() = adjusts the weights
'''
for epoch in range(EPOCHS):
    for data in trainset:
        X, y = data
        net.zero_grad()
        output = net(X.view(-1, 51).float())
        loss = F.nll_loss(output, y)
        loss.backward()
        opt.step() 
    print(loss)

tensor(1.2817, grad_fn=<NllLossBackward>)
tensor(1.2898, grad_fn=<NllLossBackward>)
tensor(1.2353, grad_fn=<NllLossBackward>)
tensor(1.0542, grad_fn=<NllLossBackward>)
tensor(1.3658, grad_fn=<NllLossBackward>)


In [223]:
#check the model
'''
no_grad() = as test data will not be used for optimization,
we do not need to calculate gradient for it
'''
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.view(-1, 51).float())
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
            print(torch.argmax(i), y[idx])

print("Accuracy: ",correct/total)

tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)


tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)


tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)


tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)


tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)


...

tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(2) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)


tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)


tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(2) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)


tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(1) tensor(5)


tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
tensor(2) tensor(5)
tensor(4) tensor(5)
tensor(3) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(2) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(1) tensor(5)
tensor(4) tensor(5)
tensor(4) tensor(5)
